In [57]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos(a):
    pos=1
    
    if a=='y':
        pos=1
        
    if a=='n':
        pos=pos*(-1)
        
    else:
        print "Please rerun the code and input either 'y' or 'n' for position"
        
    return pos

def z_factor(product1,product2):
    z=1
    factor=1000
    if product1=='RB' and product2 =='RB':
        z=42
    elif product1=='HO' and product2 =='HO':
        z=42
    elif product1=='RB' and product2 =='HO':
        z=42
    elif product1=='HO' and product2 =='RB':
        z=42
    elif product1=='G' and product2 =='G':
        factor=100
    elif product1=='G' and product2 =='B':
        z=(1/7.45)
    
    else:
        z=1
        factor=1000
    
    return z,factor
        
def create_sprd(dataframe1, dataframe2,sprd_type):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
        
    if sprd_type==1:
            
        new_df=dataframe1*42-dataframe2
        new_df['Date']=date
        new_df.set_index('Date',inplace=True)
        new_df.fillna(method='bfill',inplace=True)
        new_df.fillna(method='ffill',inplace=True)
            
        return new_df
        
        
    elif sprd_type==2:
            
        new_df= dataframe1-dataframe2
        new_df['Date']=date
        new_df.set_index('Date',inplace=True)
        new_df.fillna(method='bfill',inplace=True)
        new_df.fillna(method='ffill',inplace=True)
            
        return new_df

        
    else:
        print 'You have inputted incorrectly. Please rerun and enter either integer 1 or 2 for spread type.'
    
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i,j,loop_start,loop_end):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            
            start_date_new=dates[0]+loop_start
            end_date_new=dates[-1]+loop_end
            
            new_dates=pd.date_range(start_date_new,end_date_new)
            
            new_dates=new_dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(new_dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':new_dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date_new] # filter for start date

            temp=temp[temp['Date']<=end_date_new] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi,seasonal_year,start_date_new
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='B'
                break
            elif inst==2:
                product='G'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RB'
                break       
            elif inst==5:
                product='CL'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i,j,loop_start,loop_end):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
 
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
        
            start_date_new=dates[0]+loop_start
            end_date_new=dates[-1]+loop_end
            
            new_dates=pd.date_range(start_date_new,end_date_new)

            new_dates=new_dates.strftime('%Y-%m-%d') #make sure no left out dates

            sample_data=np.arange(0,len(new_dates)) #just sample numbers to create dataframe
            dates_dataframe=pd.DataFrame({'Date':new_dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date_new] # filter for start date

            temp=temp[temp['Date']<=end_date_new] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi,seasonal_year,end_date_new
    
    
import numpy as np
import pandas as pd
import os


%matplotlib inline

#user specify instrument and contract month

#create two lists to store the start dates and end dates - later to rename the multi-index table
sd_list=[]
ed_list=[]

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

%cd Quandl
%cd $product1
%cd $month1

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))

sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
pos_input=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()


print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

days_to_loop_start=int(raw_input('Please input the number of days to loop on start date: '))
days_to_loop_end=int(raw_input('Please input the number of days to loop on end date: '))

loop_start=-days_to_loop_start
loop_end=-days_to_loop_end

#creating the multi-index for pnl
outside=[a for a in range(loop_start,-loop_start+1)]
len_outside=len(outside)

inside=[a for a in range(loop_end,-loop_end+1)]
len_inside=len(inside)

inside=inside*len_outside

outside=outside*len_inside
outside.sort()


hier_index=list(zip(outside, inside))
hier_index=pd.MultiIndex.from_tuples(hier_index)

years_list=[]

path = os.getcwd()
listing = os.listdir(path)

for infile in listing: # get list of all years to build dataframe
    
    rd_file=pd.read_excel(infile) #read each file into panda dataframe
    yr_str=infile[-8:-4] # get the contract year
    years_list.append(yr_str)

pnl_loop=pd.DataFrame(np.random.randn(len(inside),len(years_list)),hier_index,years_list) 
##pnl multi index created

%cd ..
%cd ..
%cd ..

first_index=0
second_index=0

while first_index<=days_to_loop_start*2:
    second_index=0
    loop_end=-days_to_loop_end
    
    while second_index<=days_to_loop_end*2:
        %cd Quandl
        %cd $product1
        %cd $month1

        front_month,sy,sdn=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i,j,loop_start, loop_end)
        %cd ..
        %cd ..
        %cd ..

        %cd Quandl
        %cd $product2
        %cd $month2

        back_month,esd,edn=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i,j,loop_start, loop_end)
        %cd ..
        %cd ..
        %cd ..

        Spread_Combi=create_sprd(front_month, back_month,sprd_type)
        Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs
        Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

        position=pos(pos_input)

        zf=z_factor(product1, product2)

        #PnL Series
        pnl_series=(Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])*zf[0]*zf[1]*position
        pnl_loop.loc[loop_start].loc[loop_end]=pnl_series
        
        loop_end+=1
        second_index+=1
        ed_list.append(edn.strftime('%m-%d'))
        
    loop_start+=1
    first_index+=1
    sd_list.append(sdn.strftime('%m-%d'))
 
pnl_loop.index.names=['Start', 'End']
pnl_loop.index.set_levels([sd_list,ed_list],inplace=True)


Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 3
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 01
Please input 1 if start mth is the same as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 01
Please input end day of seasonal period in DD format: 10
Please input 1 if end mth is the same as contract year and 0 if not: 1
Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 1
Please inp

C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Doc

C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl\CL\Mar
C:\Users\boonsin.teo\Documents\Quandl\CL
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
Please rerun the code and input either 'y' or 'n' for position
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl\RB\Mar
C:\Users\boonsin.teo\Documents\Quandl\RB
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Documents
C:\Users\boonsin.teo\Documents\Quandl
C:\Users\boonsin.teo\Doc

In [58]:
pnl_loop

1996   1997   1998   1999   2000    2001   2002   2003    2004  \
Start End                                                                      
12-29 01-08   69.4  -46.2    9.0 -423.0  219.2   683.8 -103.4 -183.2  1259.4   
      01-09 -215.6 -378.0  256.0 -423.0  219.2   132.6 -279.6  831.2  2089.2   
      01-10  -90.6 -177.4  256.0 -423.0  143.2   689.0 -218.8  265.0  2089.2   
      01-11 -489.2 -177.4  256.0 -445.4 -127.0   548.8 -218.4  265.0  2089.2   
      01-12 -637.8 -177.4  123.6 -400.6  419.2   445.6 -218.4  265.0  1618.4   
12-30 01-08   56.2  -46.2  109.8 -132.6  614.2  1148.2 -103.4 -183.2   673.4   
      01-09 -228.8 -378.0  356.8 -132.6  614.2   597.0 -279.6  831.2  1503.2   
      01-10 -103.8 -177.4  356.8 -132.6  538.2  1153.4 -218.8  265.0  1503.2   
      01-11 -502.4 -177.4  356.8 -155.0  268.0  1013.2 -218.4  265.0  1503.2   
      01-12 -651.0 -177.4  224.4 -110.2  814.2   910.0 -218.4  265.0  1032.4   
12-31 01-08   56.2  -22.2  271.0 -145.6  244.2  1148.2 -103.4 -192.4  1082.6   
      01-09 -228.8 -354.0  518.0 -145.6  244.2   597.0 -279.6  822.0  1912.4   
      01-10 -103.8 -153.4  518.0 -145.6  168.2  1153.4 -218.8  255.8  1912.4   
      01-11 -502.4 -153.4  518.0 -168.0 -102.0  1013.2 -218.4  255.8  1912.4   
      01-12 -651.0 -153.4  385.6 -123.2  444.2   910.0 -218.4  255.8  1441.6   
01-01 01-08   56.2 -196.4  180.8 -252.0  244.2  1148.2 -298.0 -446.8   452.0   
      01-09 -228.8 -528.2  427.8 -252.0  244.2   597.0 -474.2  567.6  1281.8   
      01-10 -103.8 -327.6  427.8 -252.0  168.2  1153.4 -413.4    1.4  1281.8   
      01-11 -502.4 -327.6  427.8 -274.4 -102.0  1013.2 -413.0    1.4  1281.8   
      01-12 -651.0 -327.6  295.4 -229.6  444.2   910.0 -413.0    1.4   811.0   
01-02 01-08   56.2 -196.4  180.8 -252.0  244.2  1148.2 -298.0 -446.8   452.0   
      01-09 -228.8 -528.2  427.8 -252.0  244.2   597.0 -474.2  567.6  1281.8   
      01-10 -103.8 -327.6  427.8 -252.0  168.2  1153.4 -413.4    1.4  1281.8   
      01-11 -502.4 -327.6  427.8 -274.4 -102.0  1013.2 -413.0    1.4  1281.8   
      01-12 -651.0 -327.6  295.4 -229.6  444.2   910.0 -413.0    1.4   811.0   
01-03 01-08  223.4 -137.8  257.2 -252.0  244.2  1289.2 -577.2 -733.8   452.0   
      01-09  -61.6 -469.6  504.2 -252.0  244.2   738.0 -753.4  280.6  1281.8   
      01-10   63.4 -269.0  504.2 -252.0  168.2  1294.4 -692.6 -285.6  1281.8   
      01-11 -335.2 -269.0  504.2 -274.4 -102.0  1154.2 -692.2 -285.6  1281.8   
      01-12 -483.8 -269.0  371.8 -229.6  444.2  1051.0 -692.2 -285.6   811.0   
01-04 01-08   89.6 -219.6  257.2 -252.0  244.2  1181.2 -357.2 -283.2   452.0   
      01-09 -195.4 -551.4  504.2 -252.0  244.2   630.0 -533.4  731.2  1281.8   
      01-10  -70.4 -350.8  504.2 -252.0  168.2  1186.4 -472.6  165.0  1281.8   
      01-11 -469.0 -350.8  504.2 -274.4 -102.0  1046.2 -472.2  165.0  1281.8   
      01-12 -617.6 -350.8  371.8 -229.6  444.2   943.0 -472.2  165.0   811.0   

                     2005   ...      2009    2010    2011    2012    2013  \
Start End                   ...                                             
12-29 01-08  1.682200e+03   ...    3626.4  1220.8  3574.6  2009.0   308.0   
      01-09  1.682200e+03   ...    5074.4  1220.8  3574.6  2466.4  -189.6   
      01-10  1.644600e+03   ...    5074.4  1220.8  3995.4  1979.0  -362.0   
      01-11  1.405400e+03   ...    5074.4  1065.6  3450.2  2955.2 -2110.6   
      01-12  1.154400e+03   ...    6314.2  1053.4  2398.4  3336.6 -2110.6   
12-30 01-08  1.188800e+03   ...    2692.8   573.0  2704.8  1798.0   308.0   
      01-09  1.188800e+03   ...    4140.8   573.0  2704.8  2255.4  -189.6   
      01-10  1.151200e+03   ...    4140.8   573.0  3125.6  1768.0  -362.0   
      01-11  9.120000e+02   ...    4140.8   417.8  2580.4  2744.2 -2110.6   
      01-12  6.610000e+02   ...    5380.6   405.6  1528.6  3125.6 -2110.6   
12-31 01-08  2.768000e+02   ...    3096.4   655.4  2374.2  1813.6   308.0   
      01-09  2.768000e+02   ...    4544.4   655.4  2